In [ ]:
from dotenv import load_dotenv

import pandas as pd

from database.postgresql_functools import PostgreSQLManager

load_dotenv()

In [ ]:
postgres = PostgreSQLManager()

In [ ]:
query = """
    SELECT weather.*
    FROM weather
    JOIN city ON weather.city_id = city.id
    WHERE city.name = 'Canberra'
"""
df = pd.read_sql_query(query, postgres.engine)
df.show()

In [ ]:
df['RainToday'] = df['Rainfall'].apply(lambda x: 'Yes' if x > 0 else 'No')
df.show()